In [20]:
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("lightgbm")
require("ggplot2")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2101940,112.3,3690155,197.1,3598316,192.2
Vcells,12166121,92.9,445384606,3398.1,695908561,5309.4


In [21]:
#setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
setwd( "~/buckets/b1" )

kexperimento  <- "TS7318"
ksemilla_azar  <- 100019

In [22]:
dataset_finalTrain  <- fread( paste0( "./exp/", kexperimento, "/dataset_train_final.csv.gz" ) )

dataset_future  <- fread( paste0( "./exp/", kexperimento, "/dataset_future.csv.gz" ) )

In [23]:
kmax_bin           <-    31
klearning_rate     <-     0.0362216409539542

knum_iterations    <-   4055

knum_leaves        <-   1967

kmin_data_in_leaf  <-  1747

kfeature_fraction  <-    0.915840779792052
 

In [24]:
#dataset  <- fread(kdataset, stringsAsFactors= TRUE)

#paso la clase a binaria que tome valores {0,1}  enteros
#set trabaja con la clase  POS = { BAJA+1, BAJA+2 } 
#esta estrategia es MUY importante
dataset_finalTrain[ , clase01 := ifelse( clase_ternaria %in%  c("BAJA+2","BAJA+1"), 1L, 0L) ]

#los campos que se van a utilizar
campos_buenos  <- setdiff( colnames(dataset_finalTrain), c("clase_ternaria","clase01","foto_mes") )

#creo las carpetas donde van los resultados
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( paste0("./exp/", kexperimento, "/" ), showWarnings = FALSE )
setwd( paste0("./exp/", kexperimento, "/" ) )   #Establezco el Working Directory DEL EXPERIMENTO


#dejo los datos en el formato que necesita LightGBM
dtrain  <- lgb.Dataset( data= data.matrix(  dataset_finalTrain[, campos_buenos, with=FALSE]),
                        label= dataset_finalTrain[ , clase01] )

#aplico el modelo a los datos sin clase
dapply  <- dataset_future

In [25]:
dapply[,foto_mes:=NULL]

In [26]:
#genero el modelo
#estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
modelo  <- lgb.train( data= dtrain,
                      param= list( objective=          "binary",
                                   max_bin=            kmax_bin,
                                   learning_rate=      klearning_rate,
                                   num_iterations=     knum_iterations,
                                   num_leaves=         knum_leaves,
                                   min_data_in_leaf=   kmin_data_in_leaf,
                                   feature_fraction=   kfeature_fraction,
                                   seed=               ksemilla_azar
                                  )
                    )

[LightGBM] [Info] Number of positive: 5365, number of negative: 480796
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.697416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25342
[LightGBM] [Info] Number of data points in the train set: 486161, number of used features: 957
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011035 -> initscore=-4.495547
[LightGBM] [Info] Start training from score -4.495547
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [27]:
#--------------------------------------
#ahora imprimo la importancia de variables
tb_importancia  <-  as.data.table( lgb.importance(modelo) ) 
archivo_importancia  <- "predDC_impo.csv"

fwrite( tb_importancia, 
        file= archivo_importancia, 
        sep= "\t" )

In [34]:
#aplico el modelo a los datos nuevos
prediccion  <- predict( modelo, 
                        data.matrix( dapply[, campos_buenos, with=FALSE ])                                 )

#genero la tabla de entrega
tb_entrega  <-  dapply[ , list( numero_de_cliente ) ]
tb_entrega[  , prob := prediccion ]

#grabo las probabilidad del modelo
#fwrite( tb_entrega,
#        file= "prediccion.txt",
#        sep= "\t" )

#ordeno por probabilidad descendente
setorder( tb_entrega, -prob )

In [ ]:
#prob_corte <- 0.0372744903634901
#tb_entrega[  , Predicted := 0L ]
#tb_entrega[ prob >= prob_corte, Predicted := 1L ]

#  fwrite( tb_entrega[ , list(numero_de_cliente, Predicted)], 
#          file= paste0(  kexperimento, "_prob_corte.csv" ),
#          sep= "," )

In [35]:
#genero archivos con los  "envios" mejores
#deben subirse "inteligentemente" a Kaggle para no malgastar submits
cortes <- seq( 8000, 11000, by=500 )
for( envios  in  cortes )
{
  tb_entrega[  , Predicted := 0L ]
  tb_entrega[ 1:envios, Predicted := 1L ]

  fwrite( tb_entrega[ , list(numero_de_cliente, Predicted)], 
          file= paste0(  "pred_DC_",kexperimento, "_", envios, ".csv" ),
          sep= "," )
}

In [ ]:
prediccion  <- predict( modelo, 
                        data.matrix( dataset_finalTrain[, campos_buenos, with=FALSE ])                                 )

dataset_finalTrain[  , prob_baja2 := prediccion ]

#Dibujo las curvas de ganancia
setorder( dataset_finalTrain,  -prob_baja2 )

#agrego una columna que es la de las ganancias
dataset_finalTrain[ , gan :=  ifelse( clase_ternaria=="BAJA+2", 78000, -2000 ) ]
dataset_finalTrain[ , ganancia_acumulada := cumsum( gan ) ]
dataset_finalTrain[ , pos := sequence(.N) ]


In [ ]:
options(scipen=999)

##segundo grafico
ggplot(data = dataset_finalTrain[ pos <= 10000], aes(x = pos, y = ganancia_acumulada) ) +
    geom_line()